In [4]:
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Embedding, Dropout
from keras.optimizers import RMSprop
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, '../src')

from utils import *

In [6]:
## loading in multiple data sets; primarily to compare and contrast the difference betweena action and still later on
low = pd.read_csv('../stdoan-102-action-240p-20201206.csv')
med = pd.read_csv('../stdoan-102-action-480p-20201206.csv')
high = pd.read_csv('../stdoan-101-action-1080p-20201127.csv')

low = filter_ip(low)
med = filter_ip(med)
high = filter_ip(high)

In [7]:
low.head()

,Time,IP1,Port1,IP2,Port2,Proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,packet_times,packet_sizes,packet_dirs
3,1607315408,192.168.0.10,63300,137.110.0.69,443,17,23139,260613,122,214,1607315408186;1607315408201;1607315408215;1607...,612;379;126;126;193;157;165;126;165;126;482;13...,1;1;2;2;2;2;2;1;1;2;2;2;1;2;2;2;2;1;2;2;1;1;2;...
4,1607315409,192.168.0.10,63300,137.110.0.69,443,17,27301,327841,138,254,1607315409388;1607315409389;1607315409589;1607...,1362;204;1362;204;1366;505;126;1346;1346;1346;...,1;1;1;1;1;1;2;2;2;2;1;2;2;2;1;1;2;2;1;2;2;2;2;...
7,1607315410,192.168.0.10,63300,137.110.0.69,443,17,25782,393396,159,297,1607315410228;1607315410228;1607315410257;1607...,1366;381;1346;1346;1346;126;1346;1346;126;1346...,1;1;2;2;2;1;2;2;1;2;2;2;2;2;2;2;1;1;2;2;2;2;2;...
8,1607315410,192.168.0.4,55188,192.168.0.255,15600,17,63,0,1,0,1607315410867;,63;,1;
9,1607315411,192.168.0.10,63300,137.110.0.69,443,17,19908,280976,116,221,1607315411010;1607315411010;1607315411037;1607...,245;1057;126;126;206;1346;126;1346;504;126;442...,1;1;2;2;2;2;1;2;2;1;2;1;1;1;2;2;2;2;2;2;1;2;2;...


In [92]:
X_train = []
y_train = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

for i, data in enumerate([low, med, high]):
    X_train.append(list(data["1->2Bytes"][:500]))
    
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

In [93]:
model = Sequential()
model.add(LSTM(100, dropout=0.2, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(3, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [94]:
model.fit(X_train, y_train, epochs=3)

Epoch 1/3
3/3 [==============================] - 2s 591ms/step - loss: 0.7797 - accuracy: 0.6667
Epoch 2/3
3/3 [==============================] - 0s 4ms/step - loss: 0.9030 - accuracy: 0.6667
Epoch 3/3
3/3 [==============================] - 0s 4ms/step - loss: 0.9804 - accuracy: 0.3333
